In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from model import SimpleStack
from utils import check_dir
from memory import Memory
from algorithm.DQN import DQN
from environments.SimpleEnv import SimpleEnv
from utils import copy_params

In [2]:
order = "R0_1"

In [3]:
# agent view
agent_view = 7
map_size = 10
# action max
action_max = 3
# learning rate
model_save = "./model_save/"
lr = 0.002
# start play
replay_start = 20000
# update step
update_step = 1000
# gamma in q-loss calculation
gamma = 0.99
# memory pool size
memory_length = 500000
# file to save train log
summary = "./{}_Text".format(order)
# the number of step it take to linearly anneal the epsilon to it min value
annealing_end = 1000000
# min level of stochastically of policy (epsilon)-greedy
epsilon_min = 0.2
# temporary files
temporary_model = "./{}/{}.params".format(model_save,order)
temporary_pool = "./{}/{}.pool".format(model_save,order)

In [4]:
import os
if os.path.exists(summary):
    os.remove(summary)

In [5]:
ctx = mx.gpu()
for i in ["model_save", "data_save"]:
    check_dir(i)
# build models
online_model = SimpleStack(agent_view, map_size)
offline_model = SimpleStack(agent_view, map_size)
online_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().initialize(mx.init.Normal(0.02), ctx=ctx)
offline_model.collect_params().zero_grad()
print("create model")

create model


In [6]:
online_model

SimpleStack(
  (map_decode1): Sequential(
    (0): Dense(None -> 64, Activation(tanh))
    (1): Dense(None -> 64, Activation(tanh))
    (2): Dense(None -> 64, Activation(tanh))
    (3): Dense(None -> 32, Activation(tanh))
    (4): Dense(None -> 16, Activation(tanh))
  )
  (decision_making): Sequential(
    (0): Dense(None -> 3, Activation(sigmoid))
  )
)

In [7]:
# load_model = "./phase1.params"
# if os.path.exists(temporary_model):
#     online_model.load_parameters(load_model, ctx=ctx)
#     offline_model.load_parameters(load_model, ctx=ctx)
#     print("load model")

In [8]:
env = SimpleEnv(display=False)
env.reset_env()
# create pool
memory_pool = Memory(memory_length)
algorithm = DQN([online_model, offline_model], ctx, lr, gamma, memory_pool, action_max, temporary_model,bz=1024)

In [9]:
finish = 0
all_step_counter = 0
annealing_count = 0
cost = []
texts = []
num_episode = 500000
tot_reward = np.zeros(num_episode)
moving_average_clipped = 0.
moving_average = 0.

In [10]:
_epoch = 0

In [ ]:
negative_counter = 0
tmp_reward = 0
for epoch in range(_epoch, num_episode):
    _epoch += 1
    env.reset_env()
    finish = 0
    cum_clipped_reward = 0
    while not finish:
        if all_step_counter > replay_start:
            annealing_count += 1
        if all_step_counter == replay_start:
            print('annealing and learning are started')
        eps = np.maximum(1 - all_step_counter / annealing_end, epsilon_min)
        action, by = algorithm.get_action(env.state(), eps)
        old, new, reward_get, finish, original_reward = env.step(action)
        memory_pool.add(old, new, action, reward_get, finish)
        cum_clipped_reward += original_reward
        all_step_counter += 1
        if finish and len(env.finish) > 50:
            sr_50 = sum(env.finish[-50:]) / min(len(env.finish), 50)
            ar_50 = sum(env.total_reward[-50:]) / sum(env.total_step_count[-50:])
            sr_all = sum(env.finish) / len(env.finish)
            ar_all = sum(env.total_reward) / sum(env.total_step_count)
            text = "success rate last 50 %f, avg return %f; success rate total %f, avg return total %f" % (
                sr_50, ar_50, sr_all, ar_all)
            with open(summary, "a") as f:
                f.writelines(text + "\n")
            if epoch % 100 == 0:
                print(text + "; %f" % eps)
        # save model and replace online model each epoch
        if annealing_count > replay_start and annealing_count % update_step == 0:
            copy_params(offline_model, online_model)
            offline_model.save_parameters(temporary_model)
    #  train every 4 epoch
    if annealing_count > replay_start and epoch % 4 == 0:
        cost.append(algorithm.train())
    tot_reward[int(epoch) - 1] = cum_clipped_reward

success rate last 50 0.160000, avg return 0.003459; success rate total 0.207921, avg return total 0.004698; 0.995531
success rate last 50 0.100000, avg return 0.002111; success rate total 0.174129, avg return total 0.003861; 0.990937
success rate last 50 0.160000, avg return 0.003537; success rate total 0.166113, avg return total 0.003686; 0.986435
success rate last 50 0.100000, avg return 0.002186; success rate total 0.154613, avg return total 0.003410; 0.981821
annealing and learning are started
success rate last 50 0.140000, avg return 0.003038; success rate total 0.149701, avg return total 0.003290; 0.977204
success rate last 50 0.120000, avg return 0.002535; success rate total 0.149750, avg return total 0.003281; 0.972569
success rate last 50 0.140000, avg return 0.003095; success rate total 0.145506, avg return total 0.003179; 0.967913
success rate last 50 0.100000, avg return 0.002161; success rate total 0.143571, avg return total 0.003141; 0.963383
success rate last 50 0.120000

success rate last 50 0.240000, avg return 0.005484; success rate total 0.164906, avg return total 0.003626; 0.677042
success rate last 50 0.100000, avg return 0.002087; success rate total 0.163450, avg return total 0.003590; 0.672171
success rate last 50 0.140000, avg return 0.003062; success rate total 0.162717, avg return total 0.003572; 0.667445
success rate last 50 0.120000, avg return 0.002609; success rate total 0.162140, avg return total 0.003558; 0.662759
success rate last 50 0.200000, avg return 0.004556; success rate total 0.162112, avg return total 0.003557; 0.658158
success rate last 50 0.120000, avg return 0.002545; success rate total 0.161689, avg return total 0.003548; 0.653563
success rate last 50 0.260000, avg return 0.006265; success rate total 0.161667, avg return total 0.003548; 0.649057
success rate last 50 0.100000, avg return 0.002124; success rate total 0.161390, avg return total 0.003541; 0.644447
success rate last 50 0.260000, avg return 0.006217; success rate

success rate last 50 0.340000, avg return 0.008595; success rate total 0.209774, avg return total 0.004766; 0.374903
success rate last 50 0.460000, avg return 0.013707; success rate total 0.211384, avg return total 0.004808; 0.371306
success rate last 50 0.320000, avg return 0.007929; success rate total 0.212346, avg return total 0.004833; 0.367316
success rate last 50 0.280000, avg return 0.006806; success rate total 0.212606, avg return total 0.004841; 0.363160
success rate last 50 0.400000, avg return 0.010770; success rate total 0.213752, avg return total 0.004871; 0.359210
success rate last 50 0.300000, avg return 0.007339; success rate total 0.214611, avg return total 0.004893; 0.355213
success rate last 50 0.360000, avg return 0.009236; success rate total 0.215391, avg return total 0.004914; 0.351283
success rate last 50 0.280000, avg return 0.006610; success rate total 0.216160, avg return total 0.004935; 0.347331
success rate last 50 0.340000, avg return 0.008808; success rate

success rate last 50 0.480000, avg return 0.014002; success rate total 0.277170, avg return total 0.006678; 0.200000
success rate last 50 0.420000, avg return 0.011844; success rate total 0.278165, avg return total 0.006708; 0.200000
success rate last 50 0.520000, avg return 0.017544; success rate total 0.279615, avg return total 0.006752; 0.200000
success rate last 50 0.580000, avg return 0.019308; success rate total 0.280959, avg return total 0.006794; 0.200000
success rate last 50 0.520000, avg return 0.016861; success rate total 0.282153, avg return total 0.006830; 0.200000
success rate last 50 0.660000, avg return 0.022297; success rate total 0.283657, avg return total 0.006875; 0.200000
success rate last 50 0.480000, avg return 0.014414; success rate total 0.284645, avg return total 0.006905; 0.200000
success rate last 50 0.520000, avg return 0.014925; success rate total 0.285760, avg return total 0.006938; 0.200000
success rate last 50 0.500000, avg return 0.014637; success rate

success rate last 50 0.440000, avg return 0.013489; success rate total 0.350023, avg return total 0.009047; 0.200000
success rate last 50 0.720000, avg return 0.034123; success rate total 0.350970, avg return total 0.009083; 0.200000
success rate last 50 0.700000, avg return 0.028203; success rate total 0.352365, avg return total 0.009131; 0.200000
success rate last 50 0.580000, avg return 0.020611; success rate total 0.353298, avg return total 0.009165; 0.200000
success rate last 50 0.720000, avg return 0.030586; success rate total 0.354502, avg return total 0.009207; 0.200000
success rate last 50 0.500000, avg return 0.014793; success rate total 0.355455, avg return total 0.009240; 0.200000
success rate last 50 0.700000, avg return 0.029264; success rate total 0.356470, avg return total 0.009276; 0.200000
success rate last 50 0.700000, avg return 0.026820; success rate total 0.357720, avg return total 0.009319; 0.200000
success rate last 50 0.700000, avg return 0.023777; success rate

...

success rate last 50 0.980000, avg return 0.073244; success rate total 0.867491, avg return total 0.048939; 0.200000
success rate last 50 0.980000, avg return 0.080725; success rate total 0.867520, avg return total 0.048945; 0.200000
success rate last 50 0.940000, avg return 0.067821; success rate total 0.867537, avg return total 0.048948; 0.200000
success rate last 50 1.000000, avg return 0.077519; success rate total 0.867580, avg return total 0.048954; 0.200000
success rate last 50 0.960000, avg return 0.071217; success rate total 0.867593, avg return total 0.048956; 0.200000
success rate last 50 0.980000, avg return 0.073353; success rate total 0.867629, avg return total 0.048961; 0.200000
success rate last 50 0.980000, avg return 0.072808; success rate total 0.867665, avg return total 0.048967; 0.200000
success rate last 50 1.000000, avg return 0.086957; success rate total 0.867704, avg return total 0.048975; 0.200000
success rate last 50 0.960000, avg return 0.065753; success rate

success rate last 50 0.980000, avg return 0.078778; success rate total 0.869659, avg return total 0.049334; 0.200000
success rate last 50 1.000000, avg return 0.083195; success rate total 0.869700, avg return total 0.049342; 0.200000
success rate last 50 0.960000, avg return 0.073960; success rate total 0.869725, avg return total 0.049345; 0.200000
success rate last 50 0.920000, avg return 0.055758; success rate total 0.869741, avg return total 0.049349; 0.200000
success rate last 50 0.940000, avg return 0.066478; success rate total 0.869766, avg return total 0.049353; 0.200000
success rate last 50 0.940000, avg return 0.061924; success rate total 0.869791, avg return total 0.049359; 0.200000
success rate last 50 1.000000, avg return 0.082102; success rate total 0.869825, avg return total 0.049365; 0.200000
success rate last 50 0.980000, avg return 0.077409; success rate total 0.869863, avg return total 0.049372; 0.200000
success rate last 50 0.960000, avg return 0.065129; success rate

success rate last 50 0.940000, avg return 0.063257; success rate total 0.871473, avg return total 0.049682; 0.200000
success rate last 50 0.960000, avg return 0.059850; success rate total 0.871503, avg return total 0.049686; 0.200000
success rate last 50 0.920000, avg return 0.056166; success rate total 0.871518, avg return total 0.049689; 0.200000
success rate last 50 1.000000, avg return 0.086655; success rate total 0.871551, avg return total 0.049695; 0.200000
success rate last 50 0.960000, avg return 0.079734; success rate total 0.871575, avg return total 0.049700; 0.200000
success rate last 50 0.960000, avg return 0.078303; success rate total 0.871608, avg return total 0.049707; 0.200000
success rate last 50 0.940000, avg return 0.065187; success rate total 0.871617, avg return total 0.049707; 0.200000
success rate last 50 0.940000, avg return 0.062252; success rate total 0.871641, avg return total 0.049711; 0.200000
success rate last 50 0.920000, avg return 0.066282; success rate

success rate last 50 0.860000, avg return 0.049711; success rate total 0.873126, avg return total 0.049996; 0.200000
success rate last 50 0.900000, avg return 0.057841; success rate total 0.873140, avg return total 0.049999; 0.200000
success rate last 50 0.960000, avg return 0.083189; success rate total 0.873169, avg return total 0.050005; 0.200000
success rate last 50 0.980000, avg return 0.067773; success rate total 0.873192, avg return total 0.050009; 0.200000
success rate last 50 0.980000, avg return 0.076205; success rate total 0.873224, avg return total 0.050015; 0.200000
success rate last 50 0.960000, avg return 0.067416; success rate total 0.873250, avg return total 0.050020; 0.200000
success rate last 50 0.980000, avg return 0.063389; success rate total 0.873282, avg return total 0.050025; 0.200000
success rate last 50 1.000000, avg return 0.090580; success rate total 0.873317, avg return total 0.050031; 0.200000
success rate last 50 0.940000, avg return 0.067626; success rate

success rate last 50 0.980000, avg return 0.081531; success rate total 0.874816, avg return total 0.050316; 0.200000
success rate last 50 0.860000, avg return 0.050175; success rate total 0.874820, avg return total 0.050318; 0.200000
success rate last 50 0.980000, avg return 0.067961; success rate total 0.874842, avg return total 0.050322; 0.200000
success rate last 50 1.000000, avg return 0.093110; success rate total 0.874876, avg return total 0.050329; 0.200000
success rate last 50 0.960000, avg return 0.073059; success rate total 0.874898, avg return total 0.050333; 0.200000
success rate last 50 0.900000, avg return 0.062937; success rate total 0.874917, avg return total 0.050337; 0.200000
success rate last 50 0.940000, avg return 0.068713; success rate total 0.874939, avg return total 0.050342; 0.200000
success rate last 50 1.000000, avg return 0.074405; success rate total 0.874975, avg return total 0.050348; 0.200000
success rate last 50 0.980000, avg return 0.075501; success rate

In [ ]:
_number = "1"
bandwidth = 1000 # Moving average bandwidth
total_rew = np.zeros(int(_epoch)-bandwidth)
for i in range(int(_epoch)-bandwidth):
    total_rew[i] = np.sum(tot_reward[i:i+bandwidth])/bandwidth
t = np.arange(int(_epoch)-bandwidth)
belplt = plt.plot(t,total_rew[0:int(_epoch)-bandwidth],"r", label = "Return")
#handles[belplt])
plt.legend()
print('Running after %d number of episodes' %_epoch)
plt.xlabel("Number of episode")
plt.ylabel("Average Reward per episode")
plt.savefig("%s_train_%s.jpg" % (order,_number))
plt.show()
np.save("%s_train_%s.array" % (order, _number), env.total_step_count)